## Question 1

In [205]:
import datetime

import numpy as np
import pandas as pd

import scipy.optimize as opt
import pandas_datareader.data as web



In [206]:
start = datetime.datetime(1970, 1, 1)
end = datetime.datetime(2010, 12, 31)

consumption = web.DataReader('PCECC96', 'fred', start, end) 
market = web.DataReader('WILL5000INDFC', 'fred', start, end) 
t_bill = web.DataReader('TB3MS', 'fred', start, end)         
deflator = web.DataReader('GDPDEF', 'fred', start, end)      


df = pd.concat([consumption, market, deflator, t_bill], sort=True)
df = df.groupby(pd.PeriodIndex(df.index, freq='Q'), axis=0).mean() # convert to quarterly

df.columns = ['consumption', 'market', 'deflator', 't_bill']
df['rMarket'] = df.market / (df.deflator * 100)
df['rTbill'] = df.t_bill / (df.deflator * 100)


### not sure how to deflate tbill

Assume that 𝛽 = 0.99 (since this is quarterly data, this implies an annual risk- free rate of about 4%). Calculate empirical analogs of the expected values and covariance in (5). Using scipy.optimize.broyden1, solve for the value of 𝛾 that leads to equation (5) holding.


In [267]:
class GMM_Kyle( object ):   
    def __init__(self, data=df, options='q1'):
        
        self.data = data
        self.options = options
        self.gamma = None
        
        self.beta = 0.99 if self.options == 'q1' else None
        
        R = 1 / self.data.rMarket
        self.R = R[1:] # drop first value to match consumption data
        
    def estimate_m(self, gamma=None):
        ratio = self.data.consumption / np.roll(self.data.consumption, 1)
        
        # drop first obs because value has no denominator
        MUC = ratio[1:] ** -gamma     
        m = self.beta * MUC
        return m
 
    def objective(self, gamma):
        print(gamma)
        m = self.estimate_m(gamma)

        cov = np.cov(m, self.R) # var/cov matrix
        cov = cov[0,1]          # covariance of m and R
        
        
        error = cov + m.mean() * self.R[1:].mean() - 1
        return error
    
    def estimate_gamma(self, xguess=3):
        
        ### Non=linear solver is incredibly unstable, work on this
        result = opt.broyden1(self.objective, xguess)
        return result
    

In [268]:
test = GMM_Kyle(data=df, options='q1')
#test.objective(2)
test.estimate_gamma()

3.0
1.5
-619.451695902191
1.4999999953200205
-611.8836500767466
1.4999999882686843
-611.8778823972541
1.4999999812151463
-611.8757758494727
1.499999974160804
-611.8856374159421
1.4999999671102262
-611.8786923652591
1.4999999600569975
-611.8755782558079
1.4999999530025798
-611.8855274175914
1.49999994595196
-611.8756655194637
1.4999999388975755
-611.8826313823856
1.4999999318458503
-611.8716602916354
1.4999999247899365
-611.8821311857329
1.4999999177380205
-611.8815741791697
1.499999910685892
-611.8800468069037
1.4999999036331801
-611.8773518422183
1.4999998965794397
-611.8800356393122
1.4999998895267237
-611.8777187571859
1.4999998824731233
-611.8812016791512
1.4999998754208526
-611.8790543642416
1.499999868367762
-611.8769180671375
1.4999998613138559
-611.8810900888509
1.4999998542615425
-611.8827623786865
1.4999998472098675
-611.8806487405494
1.4999998401573855
-611.8773919737599
1.4999998331036604
-611.8853782701857
1.4999998260529837
-611.8806106690439
1.4999998190004873
-611.86529

NoConvergence: 1.499998598834872

In [238]:

cov(mt+1, Rt+1) + E[mt+1]* E[Rt+1] - 1 = 0
# mt+1 = beta * EMUC / MUC



SyntaxError: can't assign to operator (<ipython-input-238-33557be56f18>, line 2)

In [ ]:
df.tail()

In [166]:
df.consumption.head()

DATE
1970Q1    22.382
1970Q2    22.694
1970Q3    22.880
1970Q4    23.182
1971Q1    23.536
Freq: Q-DEC, Name: consumption, dtype: float64